In [1]:
#Import Packages used in the following analysis
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import time
import datetime

In [2]:
#Pandas Data Frame using date derived from an Excel Spreadsheet
#Print to confirm the Sheet has been read correctly
spwgt = pd.ExcelFile('SP_Weightings1.xlsx')
print(spwgt.sheet_names)

['Index Weightings']


In [3]:
#Import the data from Index Weightings Tab (Index 0) using parse
spwgt1 = spwgt.parse(0)

In [4]:
#Confirmation of the Excel read and a print out of the Data Frame head
spwgt1.info()
print(spwgt1.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 505 entries, 0 to 504
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Symbol  505 non-null    object 
 1   Weight  505 non-null    float64
dtypes: float64(1), object(1)
memory usage: 8.0+ KB
  Symbol    Weight
0   AAPL  6.115734
1   MSFT  5.635407
2   AMZN  4.301124
3     FB  2.283107
4  GOOGL  2.060597


In [5]:
#Pandas Data Frame using date derived from an CSV File
#Confirmation of the CSV read with a print out of the Data Frame head
spcons = pd.read_csv('s_p_constituents.csv')
print(spcons.head())
spcons.shape

  Symbol                 Name       Sector
0    MMM                   3M  Industrials
1    AOS          A. O. Smith  Industrials
2    ABT  Abbott Laboratories  Health Care
3   ABBV               AbbVie  Health Care
4   ABMD              Abiomed  Health Care


(505, 3)

In [6]:
#Pandas Data Frame using date derived from an Excel Spreadsheet
#Print to confirm the Sheet has been read correctly
sp500 = pd.ExcelFile('S&P_500_Daily_Returns_2020.xlsx')
print(sp500.sheet_names)

['Index Levels']


In [7]:
#Import the data from Sheet1 Tab using parse
sp500 = sp500.parse('Index Levels')

In [8]:
#Confirmation of the Excel read and a print out of the Data Frame head
sp500.info()
print(sp500.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 253 entries, 0 to 252
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Date          253 non-null    datetime64[ns]
 1   Open_Index    253 non-null    float64       
 2   High_Index    253 non-null    float64       
 3   Low_Index     253 non-null    float64       
 4   Close_Index   253 non-null    float64       
 5   Adj Close     253 non-null    float64       
 6   Volume_Index  253 non-null    int64         
dtypes: datetime64[ns](1), float64(5), int64(1)
memory usage: 14.0 KB
        Date  Open_Index  High_Index  Low_Index  Close_Index  Adj Close  \
0 2020-01-02     3244.67     3258.14    3235.53      3257.85    3257.85   
1 2020-01-03     3226.36     3246.15    3222.34      3234.85    3234.85   
2 2020-01-06     3217.55     3246.84    3214.64      3246.28    3246.28   
3 2020-01-07     3241.86     3244.91    3232.43      3237.18    3237.18   
4

In [9]:
#Step to drop the Adjusted Close price from the Data Frame as it is not required in the analysis
sp500.drop('Adj Close', axis = 1, inplace=True)

In [10]:
#Calculation to enrich the dataset with the daily movement
sp500['S&P 500 Movement'] = ((sp500['Close_Index'] / sp500['Close_Index'].shift(1))-1)*100
print(sp500.head())

        Date  Open_Index  High_Index  Low_Index  Close_Index  Volume_Index  \
0 2020-01-02     3244.67     3258.14    3235.53      3257.85    3458250000   
1 2020-01-03     3226.36     3246.15    3222.34      3234.85    3461290000   
2 2020-01-06     3217.55     3246.84    3214.64      3246.28    3674070000   
3 2020-01-07     3241.86     3244.91    3232.43      3237.18    3420380000   
4 2020-01-08     3238.59     3267.07    3236.67      3253.05    3720890000   

   S&P 500 Movement  
0               NaN  
1         -0.705987  
2          0.353339  
3         -0.280321  
4          0.490242  


In [11]:
#Setting Variables that will feed into a Web Call to Yahoo Finance to pull the daily price details for the Year 2020
#For the purposes of the exercise the user should enter Apple (AAPL) and Microsoft (MSFT) byt any S&P 500 ticker can be used
ticker1 = input('Enter Ticker 1 - ') # Enter AAPL
ticker2 = input('Enter Ticker 2 - ') # Enter MSFT
period1 = int(time.mktime(datetime.datetime(2020, 1, 1, 23, 59).timetuple()))
period2 = int(time.mktime(datetime.datetime(2020, 12, 31, 23, 59).timetuple()))
interval = '1d' # Defaulted to 1 day in this code but other frequencies are possible using '1w' for Weekly and '1m' for monthly

Enter Ticker 1 - AAPL
Enter Ticker 2 - MSFT


In [12]:
# The variables set from the preceding steps are fed into the web address so that a dynamic read of the data is possible
Stock1 = f'https://query1.finance.yahoo.com/v7/finance/download/{ticker1}?period1={period1}&period2={period2}&interval={interval}&events=history&includeAdjustedClose=true'

In [13]:
# The variables set from the preceding steps are fed into the web address so that a dynamic read of the data is possible
Stock2 = f'https://query1.finance.yahoo.com/v7/finance/download/{ticker2}?period1={period1}&period2={period2}&interval={interval}&events=history&includeAdjustedClose=true'

In [14]:
#Pandas Data Frame using date derived from the Yahoo Finance Website
#Confirmation of the Web read with a print out of the Data Frame head
#Print out the Data Frame Info to verify the date type as this will be used in the merge operation to join the tables together
Stockperiod1 = pd.read_csv(Stock1)
Stockperiod1["Date"] = pd.to_datetime(Stockperiod1["Date"])
Stockperiod1["Symbol"] = ticker1
print(Stockperiod1.head())
Stockperiod1.info()
Stockperiod1.shape

        Date       Open       High        Low      Close  Adj Close  \
0 2020-01-02  74.059998  75.150002  73.797501  75.087502  74.207466   
1 2020-01-03  74.287498  75.144997  74.125000  74.357498  73.486023   
2 2020-01-06  73.447502  74.989998  73.187500  74.949997  74.071579   
3 2020-01-07  74.959999  75.224998  74.370003  74.597504  73.723213   
4 2020-01-08  74.290001  76.110001  74.290001  75.797501  74.909149   

      Volume Symbol  
0  135480400   AAPL  
1  146322800   AAPL  
2  118387200   AAPL  
3  108872000   AAPL  
4  132079200   AAPL  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 253 entries, 0 to 252
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Date       253 non-null    datetime64[ns]
 1   Open       253 non-null    float64       
 2   High       253 non-null    float64       
 3   Low        253 non-null    float64       
 4   Close      253 non-null    float64       
 5   Adj

(253, 8)

In [15]:
#Calculation to enrich the dataset with the daily movement
Stockperiod1['{} Movement'.format(ticker1)] = ((Stockperiod1['Close'.format(ticker1)] / Stockperiod1['Close'.format(ticker1)].shift(1))-1)*100
print(Stockperiod1.head())

        Date       Open       High        Low      Close  Adj Close  \
0 2020-01-02  74.059998  75.150002  73.797501  75.087502  74.207466   
1 2020-01-03  74.287498  75.144997  74.125000  74.357498  73.486023   
2 2020-01-06  73.447502  74.989998  73.187500  74.949997  74.071579   
3 2020-01-07  74.959999  75.224998  74.370003  74.597504  73.723213   
4 2020-01-08  74.290001  76.110001  74.290001  75.797501  74.909149   

      Volume Symbol  AAPL Movement  
0  135480400   AAPL            NaN  
1  146322800   AAPL      -0.972204  
2  118387200   AAPL       0.796825  
3  108872000   AAPL      -0.470304  
4  132079200   AAPL       1.608629  


In [16]:
#Merging Data Frames to enrich the Web extract with a table of Index Constituent static data element including Name and Industry Sector
Tickermerge1 = Stockperiod1.merge(spcons, on = 'Symbol', how = 'left', suffixes = ('_{}'.format(ticker1),'_Cons'))\
.merge(spwgt1, on = 'Symbol', how = 'left', suffixes = ('_{}'.format(ticker1),'_Wgt'))
print(Tickermerge1.head())

        Date       Open       High        Low      Close  Adj Close  \
0 2020-01-02  74.059998  75.150002  73.797501  75.087502  74.207466   
1 2020-01-03  74.287498  75.144997  74.125000  74.357498  73.486023   
2 2020-01-06  73.447502  74.989998  73.187500  74.949997  74.071579   
3 2020-01-07  74.959999  75.224998  74.370003  74.597504  73.723213   
4 2020-01-08  74.290001  76.110001  74.290001  75.797501  74.909149   

      Volume Symbol  AAPL Movement   Name                  Sector    Weight  
0  135480400   AAPL            NaN  Apple  Information Technology  6.115734  
1  146322800   AAPL      -0.972204  Apple  Information Technology  6.115734  
2  118387200   AAPL       0.796825  Apple  Information Technology  6.115734  
3  108872000   AAPL      -0.470304  Apple  Information Technology  6.115734  
4  132079200   AAPL       1.608629  Apple  Information Technology  6.115734  


In [17]:
#Pandas Data Frame using date derived from the Yahoo Finance Website
#Confirmation of the Web read with a print out of the Data Frame head
#Print out the Data Frame Info to verify the date type as this will be used in the merge operation to join the tables together
Stockperiod2 = pd.read_csv(Stock2)
Stockperiod2["Date"] = pd.to_datetime(Stockperiod2["Date"])
Stockperiod2["Symbol"] = ticker2
print(Stockperiod2.head())
Stockperiod2.info()
Stockperiod2.shape

        Date        Open        High         Low       Close   Adj Close  \
0 2020-01-02  158.779999  160.729996  158.330002  160.619995  158.205765   
1 2020-01-03  158.320007  159.949997  158.059998  158.619995  156.235825   
2 2020-01-06  157.080002  159.100006  156.509995  159.029999  156.639694   
3 2020-01-07  159.320007  159.669998  157.320007  157.580002  155.211456   
4 2020-01-08  158.929993  160.800003  157.949997  160.089996  157.683731   

     Volume Symbol  
0  22622100   MSFT  
1  21116200   MSFT  
2  20813700   MSFT  
3  21634100   MSFT  
4  27746500   MSFT  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 253 entries, 0 to 252
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Date       253 non-null    datetime64[ns]
 1   Open       253 non-null    float64       
 2   High       253 non-null    float64       
 3   Low        253 non-null    float64       
 4   Close      253 non-null   

(253, 8)

In [18]:
#Calculation to enrich the dataset with the daily movement
Stockperiod2['{} Movement'.format(ticker2)] = ((Stockperiod2['Close'.format(ticker2)] / Stockperiod2['Close'.format(ticker2)].shift(1))-1)*100
print(Stockperiod2.head())

        Date        Open        High         Low       Close   Adj Close  \
0 2020-01-02  158.779999  160.729996  158.330002  160.619995  158.205765   
1 2020-01-03  158.320007  159.949997  158.059998  158.619995  156.235825   
2 2020-01-06  157.080002  159.100006  156.509995  159.029999  156.639694   
3 2020-01-07  159.320007  159.669998  157.320007  157.580002  155.211456   
4 2020-01-08  158.929993  160.800003  157.949997  160.089996  157.683731   

     Volume Symbol  MSFT Movement  
0  22622100   MSFT            NaN  
1  21116200   MSFT      -1.245175  
2  20813700   MSFT       0.258482  
3  21634100   MSFT      -0.911776  
4  27746500   MSFT       1.592838  


In [19]:
#Merging Data Frames to enrich the Web extract with a table of Index Constituent static data element including Name and Industry Sector
Tickermerge2 = Stockperiod2.merge(spcons, on = 'Symbol', how = 'left', suffixes = ('_{}'.format(ticker2),'_Cons'))\
.merge(spwgt1, on = 'Symbol', how = 'left', suffixes = ('_{}'.format(ticker2),'_Wgt'))
print(Tickermerge2.head())
Tickermerge2.shape

        Date        Open        High         Low       Close   Adj Close  \
0 2020-01-02  158.779999  160.729996  158.330002  160.619995  158.205765   
1 2020-01-03  158.320007  159.949997  158.059998  158.619995  156.235825   
2 2020-01-06  157.080002  159.100006  156.509995  159.029999  156.639694   
3 2020-01-07  159.320007  159.669998  157.320007  157.580002  155.211456   
4 2020-01-08  158.929993  160.800003  157.949997  160.089996  157.683731   

     Volume Symbol  MSFT Movement       Name                  Sector    Weight  
0  22622100   MSFT            NaN  Microsoft  Information Technology  5.635407  
1  21116200   MSFT      -1.245175  Microsoft  Information Technology  5.635407  
2  20813700   MSFT       0.258482  Microsoft  Information Technology  5.635407  
3  21634100   MSFT      -0.911776  Microsoft  Information Technology  5.635407  
4  27746500   MSFT       1.592838  Microsoft  Information Technology  5.635407  


(253, 12)

In [20]:
spmerge1 = Tickermerge1.merge(Tickermerge2, on = 'Date', how = 'left', suffixes = ('_{}'.format(ticker1),'_{}'.format(ticker2)))\
.merge(sp500, on = 'Date', how = 'left', suffixes = ('_{}'.format(ticker1),'_Index'))
print(spmerge1)
spmerge1.shape

          Date   Open_AAPL   High_AAPL    Low_AAPL  Close_AAPL  \
0   2020-01-02   74.059998   75.150002   73.797501   75.087502   
1   2020-01-03   74.287498   75.144997   74.125000   74.357498   
2   2020-01-06   73.447502   74.989998   73.187500   74.949997   
3   2020-01-07   74.959999   75.224998   74.370003   74.597504   
4   2020-01-08   74.290001   76.110001   74.290001   75.797501   
..         ...         ...         ...         ...         ...   
248 2020-12-24  131.320007  133.460007  131.100006  131.970001   
249 2020-12-28  133.990005  137.339996  133.509995  136.690002   
250 2020-12-29  138.050003  138.789993  134.339996  134.869995   
251 2020-12-30  135.580002  135.990005  133.399994  133.720001   
252 2020-12-31  134.080002  134.740005  131.720001  132.690002   

     Adj Close_AAPL  Volume_AAPL Symbol_AAPL  AAPL Movement Name_AAPL  ...  \
0         74.207466    135480400        AAPL            NaN     Apple  ...   
1         73.486023    146322800        AAPL      -

(253, 29)

In [21]:
#Simple Calcs using a for loop
total = 0
count = 0
Average = 0
maxval = 0
for tick1 in spmerge1["Close_{}".format(ticker1)]:
    total = np.round(total + tick1 ,2)
    count = count + 1
    average = np.round(total/count, 2)
print("The number of days trading is " +str(count))
print("The Average Price for the year is " + str(average))

The number of days trading is 253
The Average Price for the year is 95.35


In [22]:
#Adding a Column and if condition to compare the Primary Company daily movement to that of the Comparison Company
for ind, row in spmerge1.iterrows():
    if row['{} Movement'.format(ticker1)] < 0 and row['{} Movement'.format(ticker2)] < 0:
        spmerge1.loc[ind, "Company Movenents"] = "Both Negative"
    elif row['{} Movement'.format(ticker1)] > 0 and row['{} Movement'.format(ticker2)] > 0:
        spmerge1.loc[ind, "Company Movenents"] = "Both Positive"
    elif row['{} Movement'.format(ticker1)] > 0 and row['{} Movement'.format(ticker2)] < 0:
        spmerge1.loc[ind, "Company Movenents"] = "Company A positive to Company B"
    elif row['{} Movement'.format(ticker1)] < 0 and row['{} Movement'.format(ticker2)] > 0:
        spmerge1.loc[ind, "Company Movenents"] = "Company A Negative to Company B"
    else:
        spmerge1.loc[ind, "Company Movenents"] = "More Testing Required"

    
print(spmerge1.head())

        Date  Open_AAPL  High_AAPL   Low_AAPL  Close_AAPL  Adj Close_AAPL  \
0 2020-01-02  74.059998  75.150002  73.797501   75.087502       74.207466   
1 2020-01-03  74.287498  75.144997  74.125000   74.357498       73.486023   
2 2020-01-06  73.447502  74.989998  73.187500   74.949997       74.071579   
3 2020-01-07  74.959999  75.224998  74.370003   74.597504       73.723213   
4 2020-01-08  74.290001  76.110001  74.290001   75.797501       74.909149   

   Volume_AAPL Symbol_AAPL  AAPL Movement Name_AAPL  ...  Name_MSFT  \
0    135480400        AAPL            NaN     Apple  ...  Microsoft   
1    146322800        AAPL      -0.972204     Apple  ...  Microsoft   
2    118387200        AAPL       0.796825     Apple  ...  Microsoft   
3    108872000        AAPL      -0.470304     Apple  ...  Microsoft   
4    132079200        AAPL       1.608629     Apple  ...  Microsoft   

              Sector_MSFT  Weight_MSFT  Open_Index  High_Index  Low_Index  \
0  Information Technology     5.6

In [23]:
#Adding a Column and if condition to compare the Primary Company daily movement to that of the Index
for ind, row in spmerge1.iterrows():
    if row['{} Movement'.format(ticker1)] < 0 and row['S&P 500 Movement'] < 0:
        spmerge1.loc[ind, "Index Movenent"] = "Both Negative"
    elif row['{} Movement'.format(ticker1)] > 0 and row['S&P 500 Movement'] > 0:
        spmerge1.loc[ind, "Index Movenent"] = "Both Positive"
    elif row['{} Movement'.format(ticker1)] > 0 and row['S&P 500 Movement'] < 0:
        spmerge1.loc[ind, "Index Movenent"] = "Company A positive to Company B"
    elif row['{} Movement'.format(ticker1)] < 0 and row['S&P 500 Movement'] > 0:
        spmerge1.loc[ind, "Index Movenent"] = "Company A Negative to Company B"
    else:
        spmerge1.loc[ind, "Index Movenent"] = "More Testing Required"

    
print(spmerge1.head())

        Date  Open_AAPL  High_AAPL   Low_AAPL  Close_AAPL  Adj Close_AAPL  \
0 2020-01-02  74.059998  75.150002  73.797501   75.087502       74.207466   
1 2020-01-03  74.287498  75.144997  74.125000   74.357498       73.486023   
2 2020-01-06  73.447502  74.989998  73.187500   74.949997       74.071579   
3 2020-01-07  74.959999  75.224998  74.370003   74.597504       73.723213   
4 2020-01-08  74.290001  76.110001  74.290001   75.797501       74.909149   

   Volume_AAPL Symbol_AAPL  AAPL Movement Name_AAPL  ...  \
0    135480400        AAPL            NaN     Apple  ...   
1    146322800        AAPL      -0.972204     Apple  ...   
2    118387200        AAPL       0.796825     Apple  ...   
3    108872000        AAPL      -0.470304     Apple  ...   
4    132079200        AAPL       1.608629     Apple  ...   

              Sector_MSFT  Weight_MSFT  Open_Index  High_Index  Low_Index  \
0  Information Technology     5.635407     3244.67     3258.14    3235.53   
1  Information Technol